### Prepare SageMaker Session
please select conda_pytorch_p39 as your kernel if you are using a SageMaker Notebook Instance

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
!pip install sagemaker --upgrade

### Clone Git

In [ ]:
#!git clone https://github.com/tloen/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt

In [ ]:
#!pip uninstall bitsandbytes 
#!pip install bitsandbytes
!pip install --upgrade accelerate

### Local Training

In [ ]:
!python finetune.py \
    --base_model 'decapoda-research/llama-7b-hf' \
    --data_path 'yahma/alpaca-cleaned' \
    --output_dir '../lora-alpaca-ckpt' 

### Fine Tune on SageMaker

In [ ]:
base_model_sm = 'decapoda-research/llama-7b-hf'
data_path_sm = '/opt/ml/input/data/training'
output_dir_sm = '/opt/ml/model'

In [ ]:
hyperparams = {'base_model': base_model_sm,
                 'data_path': data_path_sm,
                 'output_dir': output_dir_sm
                 }

In [ ]:
import time
from sagemaker.pytorch import PyTorch
from sagemaker.inputs import TrainingInput
from sagemaker import get_execution_role

role = get_execution_role()
job_name = f'lora-alpaca-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

instance_count = 1

# create the Estimator
pytorch_estimator = PyTorch(
    entry_point          = 'finetune.py',          # user endpoint script
    source_dir           = '.',               # directory which includes all the files needed for training
    instance_type        = 'ml.p3.2xlarge', # instances type used for the training job
    instance_count       = instance_count,                 # the number of instances used for training
    base_job_name        = job_name,          # the name of the training job
    role                 = role,              # Iam role used in training job to access AWS ressources, e.g. S3
    framework_version    = '1.13',            # the pytorch_version version used in the training job
    py_version           = 'py39',            # the python version used in the training job
    hyperparameters      = hyperparams,
)

pytorch_estimator.fit()